# MODULE 22 - BIG DATA ANALYSIS WITH SPARK

Step 00 - Environment Setup

In [1]:
#
# Step 00 - Environment Setup
#
!pip install pyspark
import sys
import os
import shutil          as shu
import datetime
import warnings


spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
!pip install colorama

import findspark
from   colorama    import Fore
from   pyspark     import SparkFiles
from   pyspark.sql import SparkSession

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

start_time = datetime.datetime.now()
w, h       = shu.get_terminal_size()

def logStep(msg):
    l1 = len(msg)
    l2 = w - l1
    print(Fore.WHITE + str(datetime.datetime.now()) +  Fore.YELLOW + ' STEP ' + msg + Fore.WHITE + '-' * l2  )
    sys.stdout.flush()

def printSeparator():
    print(Fore.GREEN + '-' * w + Fore.WHITE)

logStep('00 - ENVIRONMENT PREPARATION')


warnings.filterwarnings('ignore')

print(F'Copyright        : {sys.copyright}')
print(F'OS Platform      : {sys.platform}')
print(F'OS Name          : {os.name}')
print(F'OS HOME          : {os.environ.get("HOME")}')
print(F'OS uName         : {os.uname().sysname}')
print(F'OS NodeName      : {os.uname().nodename}')
print(F'OS Release       : {os.uname().release}')
print(F'OS Release Ver   : {os.uname().version}')
print(F'OS Machine       : {os.uname().machine}')
print(F'Process ID       : {os.getpid()}')
print(F'Parent Process   : {os.getppid()}')
#print(F'OS User          : {os.getlogin()}')
#print(F'OS User ID       : {os.getuid()}')
#print(F'OS Group ID      : {os.getgid()}')
#print(F'OS Effective ID  : {os.geteuid()}')
#print(F'OS Effective GID : {os.getegid()}')
print(F'Current dir      : {os.getcwd()}')
print(F'Python version   : {sys.version}')
print(F'Version info     : {sys.version_info}')
print(F'Python API Ver   : {sys.api_version}')
print(F'Executable       : {sys.executable}')
print(F'Hadoop home      : {os.environ.get("HADOOP_HOME")}')
print(F'Spark version    : {findspark.__version__}')
print(F'Spark home(Find) : {findspark.find()}')
print(F'Spark Home(Env)  : {os.environ.get("SPARK_HOME")}')
print(F'Spark UI         : http://localhost:4040')
print(F'Spark submit     : {sys.argv[0]}')
print(F'Java home        : {os.environ.get("JAVA_HOME")}')

logStep("00 - DONE");
end_time           = datetime.datetime.now()
step00_elapsed_time = end_time - start_time
logStep(F"00 - ELAPSED TIME: {step00_elapsed_time} seconds")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=efe969f34ba3290b1ec693e6ed1d23dd60d1d47debd0e81551da13e7ab6f1ae6
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 https://developer.downl

Step 01 - Read in the source file into a DataFrame

In [2]:
#
# Step 01 - Read in the source file into a DataFrame.
#

start_time = datetime.datetime.now()
logStep("01 - READ SOURCE DATA");

try:
  findspark.init()
  spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
  spark.sparkContext.setLogLevel("ERROR")
  url   = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
  spark.sparkContext.addFile(url)
  home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)
  home_df.show(5)
except Exception as e:
  logStep(F"Exception: {e}")
  sys.exit(1)
else:
  logStep("01 - Spark operation - Source data read successfully")

logStep("01 - DONE");
end_time           = datetime.datetime.now()
step01_elapsed_time = end_time - start_time
logStep(F"01 - ELAPSED TIME: {step01_elapsed_time} seconds")

2023-09-06 01:19:40.867055 STEP 01 - READ SOURCE DATA-----------------------------------------------------------
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|

Step 02 - Create a temporary view of the DataFrame.

In [3]:
#
# Step 02 - Create a temporary view of the DataFrame.
#

start_time         = datetime.datetime.now()
logStep("02 - CREATE A TEMPORARY VIEW")

try:
  home_df.createOrReplaceTempView('home_sales')
except Exception as e:
  logStep(F"Exception: {e}")
  sys.exit(1)
else:
  logStep("02 - Spark operation - Temporary view created successfully")

logStep("02 - DONE")
end_time           = datetime.datetime.now()
step02_elapsed_time = end_time - start_time
logStep(F"02 - ELAPSED TIME: {step02_elapsed_time} seconds")

2023-09-06 01:20:09.650543 STEP 02 - CREATE A TEMPORARY VIEW----------------------------------------------------
2023-09-06 01:20:09.732856 STEP 02 - Spark operation - Temporary view created successfully----------------------
2023-09-06 01:20:09.735424 STEP 02 - DONE-----------------------------------------------------------------------
2023-09-06 01:20:09.736962 STEP 02 - ELAPSED TIME: 0:00:00.086202 seconds---------------------------------------


Step 03 - What is the average price for a four bedroom house sold in each year rounded to two decimal places?

In [4]:
#
# Step 03 - What is the average price for a four bedroom house sold in each year rounded to two decimal places?
#

start_time         = datetime.datetime.now()
logStep("03 - WHAT IS THE AVERAGE PRICE?=4BR")

try:
  spark.sql("SELECT YEAR(date)          AS YEAR, \
                    ROUND(AVG(price),2) AS AVERAGE \
               FROM home_sales \
              WHERE bedrooms == 4 \
           GROUP BY YEAR(date) \
           ORDER BY YEAR(date) DESC").show(5)
except Exception as e:
     logStep(F"Exception: {e}")
     sys.exit(1)
else:
     logStep("03 - Spark operation - Query executed successfully")


logStep("03 - DONE")
end_time           = datetime.datetime.now()
step03_elapsed_time = end_time - start_time
logStep(F"03 - ELAPSED TIME: {step03_elapsed_time} seconds")

2023-09-06 01:20:09.748889 STEP 03 - WHAT IS THE AVERAGE PRICE?=4BR---------------------------------------------
+----+---------+
|YEAR|  AVERAGE|
+----+---------+
|2022|296363.88|
|2021|301819.44|
|2020|298353.78|
|2019| 300263.7|
+----+---------+

2023-09-06 01:20:15.734538 STEP 03 - Spark operation - Query executed successfully------------------------------
2023-09-06 01:20:15.737727 STEP 03 - DONE-----------------------------------------------------------------------
2023-09-06 01:20:15.739796 STEP 03 - ELAPSED TIME: 0:00:05.990850 seconds---------------------------------------


Step 04 - What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

In [5]:
#
# Step 04 - What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
#

start_time         = datetime.datetime.now()
logStep("04 - WHAT IS THE AVERAGE PRICE?=3BR/3B")

try:
  spark.sql("SELECT date_built          AS YEAR , \
                    ROUND(AVG(price),2) AS AVERAGE  \
               FROM home_sales \
              WHERE bedrooms == 3 AND \
                    bathrooms == 3 \
          GROUP BY date_built \
          ORDER BY date_built DESC").show(5)
except Exception as e:
     logStep(F"Exception: {e}")
     sys.exit(1)
else:
     logStep("04 - Spark operation - Query executed successfully")


end_time           = datetime.datetime.now()
logStep("04 - DONE")
step04_elapsed_time = end_time - start_time
logStep(F"04 - ELAPSED TIME: {step04_elapsed_time} seconds")


2023-09-06 01:20:15.758704 STEP 04 - WHAT IS THE AVERAGE PRICE?=3BR/3B------------------------------------------
+----+---------+
|YEAR|  AVERAGE|
+----+---------+
|2017|292676.79|
|2016|290555.07|
|2015| 288770.3|
|2014|290852.27|
|2013|295962.27|
+----+---------+
only showing top 5 rows

2023-09-06 01:20:19.971614 STEP 04 - Spark operation - Query executed successfully------------------------------
2023-09-06 01:20:19.975483 STEP 04 - DONE-----------------------------------------------------------------------
2023-09-06 01:20:19.977222 STEP 04 - ELAPSED TIME: 0:00:04.216756 seconds---------------------------------------


Step 05 - What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors, and are greater than or equal to 2,000 square feet rounded to two decimal places?

In [6]:
#
# Step 05 - What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
#

start_time         = datetime.datetime.now()
logStep("05 - WHAT IS THE AVERAGE PRICE?=3R/3/2")

try:
  spark.sql("SELECT date_built          AS YEAR, \
                    ROUND(AVG(price),2) AS AVERAGE \
               FROM home_sales \
              WHERE bedrooms    == 3 AND \
                    bathrooms   == 3 AND \
                    floors      == 2 AND \
                    sqft_living >= 2000 \
           GROUP BY date_built \
           ORDER BY date_built DESC").show(5)
except Exception as e:
  LogStep(F"Exception: {e}")
  sys.exit(1)
else:
  logStep("05 = Spark operation - Query executed successfully")


end_time           = datetime.datetime.now()
logStep("05 - DONE")
step05_elapsed_time = end_time - start_time
logStep(F"05 - ELAPSED TIME: {step05_elapsed_time} seconds")


2023-09-06 01:20:20.004660 STEP 05 - WHAT IS THE AVERAGE PRICE?=3R/3/2------------------------------------------
+----+---------+
|YEAR|  AVERAGE|
+----+---------+
|2017|280317.58|
|2016| 293965.1|
|2015|297609.97|
|2014|298264.72|
|2013|303676.79|
+----+---------+
only showing top 5 rows

2023-09-06 01:20:22.088113 STEP 05 = Spark operation - Query executed successfully------------------------------
2023-09-06 01:20:22.091447 STEP 05 - DONE-----------------------------------------------------------------------
2023-09-06 01:20:22.092647 STEP 05 - ELAPSED TIME: 0:00:02.086773 seconds---------------------------------------


Step 06 - What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than or equal to $350,000? Although this is a small dataset, determine the run time for this query.

In [7]:
#
# Ste 06 - What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than or equal to $350,000? Although this is a small dataset, determine the run time for this query.
#

start_time         = datetime.datetime.now()
logStep("06 - WHAT IS THE AVERAGE PRICE?=300K")

try:
  spark.sql("SELECT view                AS VIEW, \
                    ROUND(AVG(price),2) AS AVERAGE \
               FROM home_sales \
           GROUP BY view \
             HAVING ROUND(AVG(price),2) >= 350000 \
           ORDER BY view DESC").show(5)
except Exception as e:
  logStep(F"Exception: {e}")
  sys.exit(1)
else:
  logStep("06 - Spark operation - Query executed successfully")

end_time           = datetime.datetime.now()
logStep("06 - DONE")
step06_elapsed_time = end_time - start_time
logStep(F"06 - ELAPSED TIME: {step06_elapsed_time} seconds")


2023-09-06 01:20:22.102053 STEP 06 - WHAT IS THE AVERAGE PRICE?=300K--------------------------------------------
+----+----------+
|VIEW|   AVERAGE|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
+----+----------+
only showing top 5 rows

2023-09-06 01:20:24.133981 STEP 06 - Spark operation - Query executed successfully------------------------------
2023-09-06 01:20:24.137354 STEP 06 - DONE-----------------------------------------------------------------------
2023-09-06 01:20:24.138486 STEP 06 - ELAPSED TIME: 0:00:02.035284 seconds---------------------------------------


Step 07 - Cache the the temporary table home_sales.

In [8]:
#
# Step 07 - Cache the the temporary table home_sales.
#

start_time         = datetime.datetime.now()
logStep("07 - CACHE HOME DATA")

try:
  spark.sql("cache table home_sales")
except Exception as e:
    logStep(F"Exception: {e}")
    sys.exit(1)
else:
    logStep("07 - Spark operation - Cache executed successfully")


end_time           = datetime.datetime.now()
logStep("07 - DONE")
step07_elapsed_time = end_time - start_time
logStep(F"07 - ELAPSED TIME: {step07_elapsed_time} seconds")

2023-09-06 01:20:24.164700 STEP 07 - CACHE HOME DATA------------------------------------------------------------
2023-09-06 01:20:27.137922 STEP 07 - Spark operation - Cache executed successfully------------------------------
2023-09-06 01:20:27.140948 STEP 07 - DONE-----------------------------------------------------------------------
2023-09-06 01:20:27.142119 STEP 07 - ELAPSED TIME: 0:00:02.976222 seconds---------------------------------------


Step 08 - Check if the table is cached.

In [9]:
#
# Step 08 - Check if the table is cached.
#

start_time         = datetime.datetime.now()
logStep("08 - IS THE DATA CACHED?")

try:
  if (spark.catalog.isCached('home_sales') == False):
    logStep("08 - home_sales is not cached")
  else:
    logStep("08 - home_sales is cached")
except Exception as e:
    logStep(F"Exception: {e}")
    sys.exit(1)
else:
    logStep("08 - Spark operation - Cache check executed successfully")

end_time           = datetime.datetime.now()
logStep("08 - DONE")
step08_elapsed_time = end_time - start_time
logStep(F"08 - ELAPSED TIME: {step08_elapsed_time} seconds")

2023-09-06 01:20:27.155680 STEP 08 - IS THE DATA CACHED?--------------------------------------------------------
2023-09-06 01:20:27.192789 STEP 08 - home_sales is cached-------------------------------------------------------
2023-09-06 01:20:27.195463 STEP 08 - Spark operation - Cache check executed successfully------------------------
2023-09-06 01:20:27.196701 STEP 08 - DONE-----------------------------------------------------------------------
2023-09-06 01:20:27.198125 STEP 08 - ELAPSED TIME: 0:00:00.041023 seconds---------------------------------------


Step 09 - Using the cached data, run the query that filters out the view ratings with average price greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

In [10]:
#
# Step 09 - Using the cached data, run the query that filters out the view ratings with average price greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
#

start_time         = datetime.datetime.now()
logStep("09 - REPEAT QUERY")

try:
  spark.sql("SELECT view                AS VIEW, \
                    ROUND(AVG(price),2) AS AVERAGE \
               FROM home_sales \
           GROUP BY view \
             HAVING ROUND(AVG(price),2) >= 350000 \
           ORDER BY view DESC").show(5)
except Exception as e:
  logStep(F"Exception: {e}")
  sys.exit(1)
else:
  logStep("09 - Spark operation - Query executed successfully")

end_time           = datetime.datetime.now()
logStep("09 - DONE")
step09_elapsed_time = end_time - start_time
logStep(F"09 - ELAPSED TIME: {step09_elapsed_time} seconds")


2023-09-06 01:20:27.206910 STEP 09 - REPEAT QUERY---------------------------------------------------------------
+----+----------+
|VIEW|   AVERAGE|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
+----+----------+
only showing top 5 rows

2023-09-06 01:20:27.840644 STEP 09 - Spark operation - Query executed successfully------------------------------
2023-09-06 01:20:27.844012 STEP 09 - DONE-----------------------------------------------------------------------
2023-09-06 01:20:27.845699 STEP 09 - ELAPSED TIME: 0:00:00.637128 seconds---------------------------------------


Step 10 - Determine the runtime and compare to the original runtime

In [11]:
#
# Step 10 Determine the runtime and compare to the original runtime
#

start_time         = datetime.datetime.now()
logStep("10 - RUNTIME DIFFERENCE")

time_difference = step06_elapsed_time - step09_elapsed_time
logStep(F"10 - Time required for a non-cached Query: {step06_elapsed_time}")
logStep(F"10 - Time required for a cached Query    : {step09_elapsed_time}")
logStep(F"10 - Time difference                     : {time_difference}")


end_time           = datetime.datetime.now()
logStep("10 - DONE")
step10_elapsed_time = end_time - start_time
logStep(F"10 - ELAPSED TIME: {step10_elapsed_time} seconds")

2023-09-06 01:20:27.860589 STEP 10 - RUNTIME DIFFERENCE---------------------------------------------------------
2023-09-06 01:20:27.863880 STEP 10 - Time required for a non-cached Query: 0:00:02.035284-----------------------
2023-09-06 01:20:27.866016 STEP 10 - Time required for a cached Query    : 0:00:00.637128-----------------------
2023-09-06 01:20:27.867402 STEP 10 - Time difference                     : 0:00:01.398156-----------------------
2023-09-06 01:20:27.870299 STEP 10 - DONE-----------------------------------------------------------------------
2023-09-06 01:20:27.871701 STEP 10 - ELAPSED TIME: 0:00:00.009663 seconds---------------------------------------


Step 11 - Partition by the "date_built" field on the formatted parquet home sales data

In [12]:
#
# Step 11 - Partition by the "date_built" field on the formatted parquet home sales data
#

start_time         = datetime.datetime.now()
logStep("11 - FORMATTED PARQUET")

try:
  home_df.write.parquet('home_parquet', mode='overwrite',partitionBy='date_built')
  home_df.show(5)
except Exception as e:
  logStep(F"Exception: {e}")
  sys.exit(1)
else:
  logStep("11 - Spark operation - Parquet file created successfully.")

end_time           = datetime.datetime.now()
logStep("11 - DONE")
step11_elapsed_time = end_time - start_time
logStep(F"11 - ELAPSED TIME: {step11_elapsed_time} seconds")

2023-09-06 01:20:27.903802 STEP 11 - FORMATTED PARQUET----------------------------------------------------------
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|

Step 12 - Read the parquet formatted data.

In [13]:
#
# Step 12 - Read the parquet formatted data.
#

start_time         = datetime.datetime.now()
logStep("12 - READ PARQUET")

try:
  parquet_home_df = spark.read.parquet('home_parquet')
  parquet_home_df.show(5)
except Exception as e:
  logStep(Fore.RED + F"Exception: {e}")
  sys.exit(1)
else:
  logStep("12 - Spark operation - Parquet file read successfully")

end_time           = datetime.datetime.now()
logStep("12 - DONE")
step12_elapsed_time = end_time - start_time
logStep(F"12 - ELAPSED TIME: {step12_elapsed_time} seconds")

2023-09-06 01:20:32.535191 STEP 12 - READ PARQUET---------------------------------------------------------------
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|  

Step 13 - Create a temporary table for the parquet data.

In [14]:
#
# Step 13 - Create a temporary table for the parquet data.
#

start_time         = datetime.datetime.now()
logStep("13 - CREATE PARQUET VIEW")

try:
  parquet_home_df.createOrReplaceTempView('parquet_temp_home')
  parquet_home_df.show(5)
except Exception as e:
  print(Fore.RED + F"Exception: {e}")
  sys.exit(1)
else:
  logStep("13 - Spark operation - Parquet view created successfully")

end_time           = datetime.datetime.now()
logStep("13 - DONE")
step13_elapsed_time = end_time - start_time
logStep(F"13 - ELAPSED TIME: {step13_elapsed_time} seconds")

2023-09-06 01:20:34.385870 STEP 13 - CREATE PARQUET VIEW--------------------------------------------------------
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|  

Step 14 - Run the query that filters out the view ratings with average price of greater than or equal to $350,000 with the parquet DataFrame. Round your average to two decimal places

In [15]:
#
# Step 14 - Run the query that filters out the view ratings with average price of greater than or equal to $350,000 with the parquet DataFrame. Round your average to two decimal places.
#

start_time         = datetime.datetime.now()
logStep("14 - REPEAT QUERY")

try:
  spark.sql("SELECT view                AS VIEW, \
                    ROUND(AVG(price),2) AS AVERAGE \
               FROM parquet_temp_home \
           GROUP BY view \
             HAVING ROUND(AVG(price),2) >= 350000 \
           ORDER BY view DESC").show(5)
except Exception as e:
  logStep(Fore.RED + F"Exception: {e}")
  sys.exit(1)
else:
  logStep("14 - Spark operation - Parquet query executed successfully.")

end_time           = datetime.datetime.now()
logStep("14 - DONE")
step14_elapsed_time = end_time - start_time
logStep(F"14 - ELAPSED TIME: {step14_elapsed_time} seconds")

2023-09-06 01:20:35.018483 STEP 14 - REPEAT QUERY---------------------------------------------------------------
+----+----------+
|VIEW|   AVERAGE|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
+----+----------+
only showing top 5 rows

2023-09-06 01:20:36.260975 STEP 14 - Spark operation - Parquet query executed successfully.---------------------
2023-09-06 01:20:36.271403 STEP 14 - DONE-----------------------------------------------------------------------
2023-09-06 01:20:36.281185 STEP 14 - ELAPSED TIME: 0:00:01.252779 seconds---------------------------------------


Step 15 Determine the runtime and compare to the original runtime

In [16]:
#
# Step 15 - Determine the runtime and compare it to the cached version.
#

start_time          = datetime.datetime.now()
logStep("15 - RUNTIME DIFFERENCE")

time_difference     = step06_elapsed_time - step14_elapsed_time
logStep(F"15 - Time required for a non-cached Query    : {step06_elapsed_time}")
logStep(F"15 - Time required for a parquet cached Query: {step14_elapsed_time}")
logStep(F"15 - Time difference                         : {time_difference}")
logStep("15 - DONE")
end_time            = datetime.datetime.now()
step15_elapsed_time = end_time - start_time
logStep(F"15 - ELAPSED TIME: {step15_elapsed_time} seconds")

2023-09-06 01:20:36.291405 STEP 15 - RUNTIME DIFFERENCE---------------------------------------------------------
2023-09-06 01:20:36.293653 STEP 15 - Time required for a non-cached Query    : 0:00:02.035284-------------------
2023-09-06 01:20:36.296898 STEP 15 - Time required for a parquet cached Query: 0:00:01.252779-------------------
2023-09-06 01:20:36.298530 STEP 15 - Time difference                         : 0:00:00.782505-------------------
2023-09-06 01:20:36.300586 STEP 15 - DONE-----------------------------------------------------------------------
2023-09-06 01:20:36.301991 STEP 15 - ELAPSED TIME: 0:00:00.010490 seconds---------------------------------------


Step 16 - Un-cache the home_sales temporary table.

In [17]:
#
# Step 16 - Un-cache the home_sales temporary table.
#

start_time         = datetime.datetime.now()
logStep("16 - UNCACHE")

try:
  spark.sql("uncache table home_sales")
except Exception as e:
  logStep(Fore.RED + F"Exception: {e}")
  sys.exit(1)
else:
  logStep("16 - Spark operation - Uncache executed successfully")

end_time           = datetime.datetime.now()
logStep("16 - DONE")
step16_elapsed_time = end_time - start_time
logStep(F"16 - ELAPSED TIME: {step16_elapsed_time} seconds")

2023-09-06 01:20:36.312424 STEP 16 - UNCACHE--------------------------------------------------------------------
2023-09-06 01:20:36.410382 STEP 16 - Spark operation - Uncache executed successfully----------------------------
2023-09-06 01:20:36.413409 STEP 16 - DONE-----------------------------------------------------------------------
2023-09-06 01:20:36.414744 STEP 16 - ELAPSED TIME: 0:00:00.100961 seconds---------------------------------------


Step 17 - Check if the home_sales is no longer cached

In [18]:
#
# Step 17 - Check if the home_sales is no longer cached
#
start_time         = datetime.datetime.now()
logStep("17 - CACHE CHECK")

try:
  if (spark.catalog.isCached('home_sales') == False):
      logStep("17 - home_sales is not cached")
  else:
      logStep("17 - home_sales is cached")
except Exception as e:
  logStep(F"Exception: {e}")
  sys.exit(1)
else:
  logStep("17 - Spark operation - Cache check executed successfully")

end_time           = datetime.datetime.now()
logStep("17 - DONE")
step17_elapsed_time = end_time - start_time
logStep(F"17 - ELAPSED TIME: {step17_elapsed_time} seconds")


2023-09-06 01:20:36.425238 STEP 17 - CACHE CHECK----------------------------------------------------------------
2023-09-06 01:20:36.459050 STEP 17 - home_sales is not cached---------------------------------------------------
2023-09-06 01:20:36.462025 STEP 17 - Spark operation - Cache check executed successfully------------------------
2023-09-06 01:20:36.463631 STEP 17 - DONE-----------------------------------------------------------------------
2023-09-06 01:20:36.465290 STEP 17 - ELAPSED TIME: 0:00:00.038359 seconds---------------------------------------


In [19]:
logStep("TOTAL RUN TIME")
logStep(F" 0:   {step00_elapsed_time} seconds")
logStep(F" 1:   {step01_elapsed_time} seconds")
logStep(F" 2:   {step02_elapsed_time} seconds")
logStep(F" 3:   {step03_elapsed_time} seconds")
logStep(F" 4:   {step04_elapsed_time} seconds")
logStep(F" 5:   {step05_elapsed_time} seconds")
logStep(F" 6:   {step06_elapsed_time} seconds - non-cached query")
logStep(F" 7:   {step07_elapsed_time} seconds")
logStep(F" 8:   {step08_elapsed_time} seconds")
logStep(F" 9:   {step09_elapsed_time} seconds - cached query")
logStep(F"10:   {step10_elapsed_time} seconds")
logStep(F"11:   {step11_elapsed_time} seconds")
logStep(F"12:   {step12_elapsed_time} seconds")
logStep(F"13:   {step13_elapsed_time} seconds")
logStep(F"14:   {step14_elapsed_time} seconds - parquet cached query")
logStep(F"15:   {step15_elapsed_time} seconds")
logStep(F"16:   {step16_elapsed_time} seconds")
logStep(F"17:   {step17_elapsed_time} seconds")
logStep(F"TT:   {step00_elapsed_time + step01_elapsed_time + step02_elapsed_time + step03_elapsed_time + step04_elapsed_time + step05_elapsed_time + step06_elapsed_time + step07_elapsed_time + step08_elapsed_time + step09_elapsed_time + step10_elapsed_time + step11_elapsed_time + step12_elapsed_time + step13_elapsed_time + step14_elapsed_time + step15_elapsed_time + step16_elapsed_time + step17_elapsed_time} seconds")
logStep(F'06 Cached Query Reduction  : {100-(step09_elapsed_time/step06_elapsed_time*100):.2f}%')
logStep(F'14 Parquet Query Reduction : {100-(step14_elapsed_time/step06_elapsed_time*100):.2f}%')
logStep("END OF PROGRAM-")

2023-09-06 01:20:36.478884 STEP TOTAL RUN TIME------------------------------------------------------------------
2023-09-06 01:20:36.490187 STEP  0:   0:00:00.007319 seconds----------------------------------------------------
2023-09-06 01:20:36.491488 STEP  1:   0:00:28.770066 seconds----------------------------------------------------
2023-09-06 01:20:36.492820 STEP  2:   0:00:00.086202 seconds----------------------------------------------------
2023-09-06 01:20:36.493913 STEP  3:   0:00:05.990850 seconds----------------------------------------------------
2023-09-06 01:20:36.495234 STEP  4:   0:00:04.216756 seconds----------------------------------------------------
2023-09-06 01:20:36.496366 STEP  5:   0:00:02.086773 seconds----------------------------------------------------
2023-09-06 01:20:36.497384 STEP  6:   0:00:02.035284 seconds - non-cached query---------------------------------
2023-09-06 01:20:36.498429 STEP  7:   0:00:02.976222 seconds------------------------------------